<a href="https://colab.research.google.com/github/EvandroOCosta/scripts/blob/main/CANDELAMENTOS_STREAMING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ANÁLISE DE DADOS PARA INDENTIFICAR OS MOTIVOS DE CANCELAMENTO DAS ASSINATURAS DA STREAMING

IDENTIFICANDO O PROBLEMA: Ao longo do tempo, notou-se que o número de cancelamentos de assinaturas da plataforma de streaming tem aumentado.

Este script tem o objetive de analisar os motivos de cancelamento e propor medidas estratégicas para reduzir a evasão.

In [1]:
# PASSO 1 - importar base de dados
# bibliotecas:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [5]:
# importando DataSet
df = pd.read_csv('/content/cancelamentos.csv')
df = df.drop(columns='CustomerID') # eliminando colunas desnecessárias para a análise
display(df)

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
881661,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
881662,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
881663,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
881664,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [ ]:
# PASSO 2 - Analisar a base de dados
df.info() # verificar a existência de valores nulos

In [3]:
# PASSO 3 - limpar a base de dados
# missing values
df = df.dropna() # como a quantidade de linhas com valores nulos é baixa, é viável excuí-los do data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151253 entries, 0 to 151257
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idade                   151253 non-null  float64
 1   sexo                    151253 non-null  object 
 2   tempo_como_cliente      151253 non-null  float64
 3   frequencia_uso          151253 non-null  float64
 4   ligacoes_callcenter     151253 non-null  float64
 5   dias_atraso             151253 non-null  float64
 6   assinatura              151253 non-null  object 
 7   duracao_contrato        151253 non-null  object 
 8   total_gasto             151253 non-null  float64
 9   meses_ultima_interacao  151253 non-null  float64
 10  cancelou                151253 non-null  float64
dtypes: float64(8), object(3)
memory usage: 13.8+ MB


In [6]:
# VERIFICANDO A QUANTIDADE DE CANCELAMENTOS EM VALORES ABSOLUTOS E EM PERCENTUAL;
display(df['cancelou'].value_counts().map('{:,}'.format))                       # valores absolutos
display(df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format))       # percentual

1.0    499,998
0.0    381,666
Name: cancelou, dtype: object

1.0    56.7%
0.0    43.3%
Name: cancelou, dtype: object

LEGENDA: 1.0 - CLIENTES QUE CANCELARAM
         0.0 - CLIENTES QUE NÃO CANCELARAM

OBJETIVO: PROPOR MEDIDAS PARA A REDUÇÃO DESSE PERCENTUAL DE CANCELAMENTOS

In [ ]:
# MONTANDO UMA MATRIZ DE CORRELAÇÃO ENTRE AS COLUNAS
df.corr()

<ipython-input-30-223a48fd12f9>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
idade,1.000000,-0.011629,-0.007190,0.158451,0.061738,-0.084684,0.028980,0.218394
tempo_como_cliente,-0.011629,1.000000,-0.026799,-0.027642,-0.016586,0.019007,-0.006904,-0.051920
frequencia_uso,-0.007190,-0.026799,1.000000,-0.022013,-0.014470,0.018631,-0.004662,-0.046101
ligacoes_callcenter,0.158451,-0.027642,-0.022013,1.000000,0.162889,-0.221594,0.077684,0.574267
dias_atraso,0.061738,-0.016586,-0.014470,0.162889,1.000000,-0.121044,0.042708,0.312129
total_gasto,-0.084684,0.019007,0.018631,-0.221594,-0.121044,1.000000,-0.056890,-0.429355
meses_ultima_interacao,0.028980,-0.006904,-0.004662,0.077684,0.042708,-0.056890,1.000000,0.149616
cancelou,0.218394,-0.051920,-0.046101,0.574267,0.312129,-0.429355,0.149616,1.000000


In [ ]:
# CRIANDO UM GRÁFICO DE MAPA DE CALOR DA CORRELAÇÃO ENTRE AS COLUNAS DA BASE DE DADOS
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, fmt='.2f', ax=ax)
plt.show()

In [ ]:
# PASSO 5 - ANALISAR AS CAUSAS
# CRIANDO UM GRÁFICO DE RELAÇÃO ENTRE O NÚMERO DE CANCELAMENTOS E CADA COLUNA SEPARADAMENTE
for coluna in df.columns:
  grafico = px.histogram(df,x=coluna,color='cancelou')
  grafico.update_layout(bargap=0.2)
  grafico.show()


DIAGNÓSTICO:

Foram detectados 3 gruppos de clientes que efeturam o cancelamento das assinaturas:

- Clientes que acionaram o Call Center mais de 4 vezes;
- Clientes com atraso no pagamento maior de 20 dias;
- Clientes adeptos ao plano mensal.

CENÁRIO ATUAL: 56,7% dos clientes fizeram o cancelamento

GRUPO 1 - CLIENTES QUE ACIONARAM O CALL CENTRE MAIS DE 4 VAZES

*** SUGESTÃO: aprimorar o atendimento ao cliente para que suas solicitações sejam atendidas com menos interações.

SIMULANDO O CENÁRIO COM A QUESTÃO DO CALL CENTER RESOLVIDA:

In [7]:
# ELIMINANDO OS CANCELAMENTOS SEM OS CLIENTES QUE ACIONARAM MAIS DE 4 VEZES O CALL CENTER
df = df[df['ligacoes_callcenter']<=4]
display(df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format))


0.0    63.7%
1.0    36.3%
Name: cancelou, dtype: object

O percentual de cancelamentos reduzi para 36,3%!!

Todos os planos mensais foram cancelados.
SUGESTÃO: promover promoções nos planos anual e trimestral, para reduzir as assinaturas mensais

SIMULANDO O CENÁRIO COM AS QUESTÕES DO CALL CENTER E TEMPO DO PLANO RESOLVIDAS:

In [8]:
# ELIMINANDO DA AMOSTRA OS PLANOS MENSAIS CANSELADOS
df = df[df['duracao_contrato'] != 'Monthly']
display(df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format))

0.0    73.6%
1.0    26.4%
Name: cancelou, dtype: object

NOVA REDUÇÃO: DE 36,3% PARA 26,4%

Clientes com mais de 20 dias de atraso tende a cancelar o serviço.
SUGESTÃO: aprimorar as formas de pagamento e cobrança

In [9]:
# ELIMINANDO DA AMOSTRA OS CANCELAMENTOS COM ATRASO DE PAGAMENTO SUPERIOR A 20 DIAS
df = df[df['dias_atraso'] <= 10]
display(df['cancelou'].value_counts(normalize=True).map('{:.1%}'.format))

0.0    81.6%
1.0    18.4%
Name: cancelou, dtype: object

NOVA REDUÇÃO DO PERCENTUAL DE CANCELAMENTOS: 18,4%!!

ANÁLISE DE DADOS - CONHEÇA SEU NEGÓCIO, SEUS DADOS E TENHA INSIGHTS VALIOSOS PARA AS SUAS DECISÕES ESTRATÉGICAS.

In [11]:
classificar = pd.read_csv('/content/Pasta de Trabalho1.csv',sep=';')

classificar = classificar.drop(columns='CustomerID') # eliminando colunas desnecessárias para a análise
classificar = classificar.dropna()

display(classificar)

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0
...,...,...,...,...,...,...,...,...,...,...
95,26.0,Male,15.0,11.0,0.0,20.0,Basic,Annual,269.0,1.0
96,58.0,Female,28.0,17.0,0.0,24.0,Premium,Quarterly,820.0,29.0
97,56.0,Female,16.0,11.0,5.0,18.0,Premium,Monthly,590.0,27.0
98,42.0,Female,12.0,6.0,8.0,25.0,Basic,Monthly,566.0,11.0
